## 網頁爬取CoinDesk文章

In [1]:
# 載入需要的套件
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
import random
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import random



## 爬取文章內連結

In [22]:
#瀏覽器設定，無痕、Fake Agent
opts = Options()
opts.add_argument("--incognito")
ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
opts.add_argument("user-agent={}".format(ua))


# CoinURL:爬取到的網址
CoinURL=[]

# CoinType:爬取到文章類型
CoinType=[]

#設定使用瀏覽器、最大化
driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
driver.get("https://www.coindesk.com/search?s=bitcoin&sort=1")
driver.maximize_window()


for i in range(1000): 
    driver.implicitly_wait(6) 
    soup = BeautifulSoup(driver.page_source, 'html5lib')   
    allnode_of_a = soup.find_all("a",class_='Box-sc-1hpkeeg-0 hBnhmi')
    result = [_.get("href") for _ in allnode_of_a]
    # Context_type=[_.split('/')[1] for _ in result]
    CoinURL=CoinURL+result
    # CoinType=CoinType+Context_type


    driver.execute_script("window.scrollBy(0, 2500);")

    #使用按鍵到下一頁
    if i ==1:Next=driver.find_element(By.XPATH,'//*[@id="queryly_advanced_container"]/div[5]/div[2]/button[5]/h6')
    else:Next=driver.find_element(By.XPATH,'//*[@id="queryly_advanced_container"]/div[5]/div[2]/button[4]/h6')
    time.sleep(random.randrange(5))
    driver.execute_script("arguments[0].click();", Next)

    if (i%100==0) & (i>0) :
        lo={'CoinURL':CoinURL}
        df = pd.DataFrame(lo)
        df.to_csv('Data_URL/CoinURL-'+str(i)+'.csv')
        print(str(i)+' times done')

lo={'CoinURL':CoinURL}
df = pd.DataFrame(lo)
df.to_csv('Data_URL/CoinURL.csv')

# 關閉瀏覽器視窗
driver.close() 


C:\Users\nptu\AppData\Local\Temp\ipykernel_11184\2466462473.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
C:\Users\nptu\AppData\Local\Temp\ipykernel_11184\2466462473.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)


100 times done
200 times done
300 times done
400 times done
500 times done
600 times done
700 times done
800 times done
900 times done


## 在網址取得後，爬取文章內容

In [8]:
from selenium.webdriver.chrome.options import Options

opts = Options()
opts.add_argument("--incognito")  # 使用無痕模式。用 selenium開瀏覽器已經很乾淨了，但疑心病重的可以用一下
ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
opts.add_argument("user-agent={}".format(ua))  



df = pd.read_csv('Data_URL\CoinURL_final.csv')
driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
CoinText=[]
for url in range(len(df['CoinURL'])):
    try:
        driver.implicitly_wait(15)
        driver.refresh()        
        driver.get("https://www.coindesk.com"+df['CoinURL'][url]) # 更改網址以前往不同網頁
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        allnode_of_a = soup.find_all("div",class_='common-textstyles__StyledWrapper-sc-18pd49k-0 eSbCkN')


        result=["".join([_.getText() for _ in allnode_of_a])]
        CoinText+=result
        print('T'+str(url))
    except:
        driver.refresh()
        driver.get("https://www.coindesk.com"+df['CoinURL'][url]) # 更改網址以前往不同網頁
        driver.implicitly_wait(15)
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        allnode_of_a = soup.find_all("div",class_='common-textstyles__StyledWrapper-sc-18pd49k-0 eSbCkN')
        result=["".join([_.getText() for _ in allnode_of_a])]
        CoinText+=result
        print('F'+str(url))
    if ((url%100)==0) &(url>0):
        driver.close() 
        lo={'CoinText':CoinText,'CoinDate':df['CoinDate'][16402:url+1]}
        df_N = pd.DataFrame(lo)
        df_N.to_csv('CoinText_'+str(url)+'.csv')
        df_N.to_csv('CoinText_final.csv')
        driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)

lo={'CoinText':CoinText,'CoinDate':df['CoinDate']}
df_N = pd.DataFrame(lo)
df_N.to_csv('CoinText_final.csv')

driver.close() # 關閉瀏覽器視窗

C:\Users\nptu\AppData\Local\Temp\ipykernel_11736\2179663412.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)
C:\Users\nptu\AppData\Local\Temp\ipykernel_11736\2179663412.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver.exe',chrome_options=opts)


T0
T1
T2
T3
T4
T5
T6
T7
T8
T9
T10
T11
T12
T13
T14
T15
T16
T17
T18
T19
T20
T21
T22
T23
T24
T25


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D58893+48451]
	(No symbol) [0x00CEB8A1]
	(No symbol) [0x00BF5058]
	(No symbol) [0x00BE7BB6]
	(No symbol) [0x00BE78D9]
	(No symbol) [0x00BF6460]
	(No symbol) [0x00C4C038]
	(No symbol) [0x00C3A0B6]
	(No symbol) [0x00C17E08]
	(No symbol) [0x00C18F2D]
	GetHandleVerifier [0x00FB8E3A+2540266]
	GetHandleVerifier [0x00FF8959+2801161]
	GetHandleVerifier [0x00FF295C+2776588]
	GetHandleVerifier [0x00DE2280+612144]
	(No symbol) [0x00CF4F6C]
	(No symbol) [0x00CF11D8]
	(No symbol) [0x00CF12BB]
	(No symbol) [0x00CE4857]
	BaseThreadInitThunk [0x76C200F9+25]
	RtlGetAppContainerNamedObjectPath [0x77567BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77567B8E+238]


## 在網址取得後，爬取文章內容BeautifulSoup

In [18]:
df = pd.read_csv('Data_URL\CoinURL_final.csv')
CoinText=[]
for url in range(len(df['CoinURL'])):
    time.sleep(random.randint(3,9))
    response = requests.get("https://www.coindesk.com"+df['CoinURL'][url])
    soup = BeautifulSoup(response.text, "html.parser")
    
    allnode_of_a = soup.find_all("div",class_='common-textstyles__StyledWrapper-sc-18pd49k-0 eSbCkN')
    result=["".join([_.getText() for _ in allnode_of_a])]
    CoinText+=result
    print('T'+str(url))

    if ((url%100)==0) &(url>0):
        lo={'CoinText':CoinText,'CoinDate':df['CoinDate'][0:url+1]}
        df_N = pd.DataFrame(lo)
        df_N.to_csv('CoinText_'+str(url)+'.csv')
        df_N.to_csv('CoinText_final.csv')

lo={'CoinText':CoinText,'CoinDate':df['CoinDate']}
df_N = pd.DataFrame(lo)
df_N.to_csv('CoinText_final.csv')


T0
T1
T2
T3
T4
T5
T6
T7
T8
T9
T10
T11
T12
T13
T14
T15
T16
T17
T18
T19
T20
T21
T22
T23
T24
T25
T26
T27
T28
T29
T30
T31
T32
T33
T34
T35
T36
T37
T38
T39
T40
T41
T42
T43
T44
T45
T46
T47
T48
T49
T50
T51
T52
T53
T54
T55
T56
T57
T58
T59
T60
T61
T62
T63
T64
T65
T66
T67
T68
T69
T70
T71
T72
T73
T74
T75
T76
T77
T78
T79
T80
T81
T82
T83
T84
T85
T86
T87
T88
T89
T90
T91
T92
T93
T94
T95
T96
T97
T98
T99
T100
T101
T102
T103
T104
T105
T106
T107
T108
T109
T110
T111
T112
T113
T114
T115
T116
T117
T118
T119
T120
T121
T122
T123
T124
T125
T126
T127
T128
T129
T130
T131
T132
T133
T134
T135
T136
T137
T138
T139
T140
T141
T142
T143
T144
T145
T146
T147
T148
T149
T150
T151
T152
T153
T154
T155
T156
T157
T158
T159
T160
T161
T162
T163
T164
T165
T166
T167
T168
T169
T170
T171
T172
T173
T174
T175
T176
T177
T178
T179
T180
T181
T182
T183
T184
T185
T186
T187
T188
T189
T190
T191
T192
T193
T194
T195
T196
T197
T198
T199
T200
T201
T202
T203
T204
T205
T206
T207
T208
T209
T210
T211
T212
T213
T214
T215
T216
T217
T218
T219
T220
T221


In [2]:
df = pd.read_csv('Data_URL\CoinURL_final.csv')
df.shape

(3482, 4)